In [1]:
import pymysql
import pandas as pd

아래 2개의 테이블을 생성하고, 주어진 문제에 대한 sql문장을 작성하시오.

In [2]:
conn = pymysql.connect(host='localhost', user='root', password='102377', 
db = 'shoppingmall', charset='utf8')

list_user = [
    ['KHD', '강호동', 1970, '경북', '011', '22222222', 187, '2007-07-07'],
    ['KJD', '김제동', 1974, '경남', '', '', 173, '2013-03-03'],
    ['KKJ', '김국진', 1965, '서울', '019', '33333333', 171 ,'2009-09-09'],
    ['KYM', '김용만', 1972, '서울', '010', '44444444', 177 ,'2015-05-05'],
    ['LHJ', '이휘재', 1972, '경기', '011', '88888888', 180 ,'2006-04-04'],
    ['LKK', '이경규', 1960, '경남', '018', '99999999', 170 ,'2004-12-12'],
    ['NHS', '남희석', 1971, '충남', '016', '66666666', 180 ,'2017-04-04'],
    ['PSH', '박수홍', 1970, '서울', '010', '00000000', 183 ,'2012-05-05'],
    ['SDY', '신동엽', 1971, '경기', '', '', 176 ,'2008-10-10'],
    ['YJS', '유재석', 1972, '서울', '010', '11111111', 178 ,'2008-08-08']
]

list_buy = [
    ['KHD', '운동화', '', 30, 2],
    ['KHD', '노트북', '전자', 1000, 1],
    ['KYM', '모니터', '전자', 200, 1],
    ['PSH', '모니터', '전자', 200, 5],
    ['KHD', '청바지', '의류', 50, 1],
    ['PSH', '메모리', '전자', 80, 10],
    ['KJD', '책', '서적', 15, 5],
    ['LHJ', '책', '서적', 15, 2],
    ['LHJ', '청바지', '의류', 50, 1],
    ['PSH', '운동화', '', 30, 2],
    ['LHJ', '책', '서적', 15, 1],
    ['PSH', '운동화', '', 30, 2],
]

In [3]:
cur = conn.cursor()

sql_user_insert = """insert into user_table(userID, userName, birthYear, addr, mobile1, mobile2, height, mDate)
values (%s, %s, %s, %s, %s, %s, %s, %s)"""

for num in range(len(list_user)) : 
    cur.execute(sql_user_insert, (list_user[num][0], list_user[num][1], list_user[num][2], list_user[num][3],
    list_user[num][4],list_user[num][5],list_user[num][6],list_user[num][7]))


In [4]:
sql_buy_insert = """insert into buy_table(userID, prodName, groupName, price, amount)
values (%s, %s, %s, %s, %s)"""

for num in range(len(list_buy)) : 
    cur.execute(sql_buy_insert, (list_buy[num][0], list_buy[num][1], list_buy[num][2], list_buy[num][3],
    list_buy[num][4]))
    
conn.commit()

In [5]:
conn.close()

내부 조인한 결과에 ‘연락처’ 컬럼 추가

In [6]:
conn = pymysql.connect(host='localhost', user='root', password='102377', 
db = 'shoppingmall', charset='utf8')
cur = conn.cursor()

In [9]:
sql_01 = """select u.userName, b.prodName, u.addr, CONCAT(u.mobile1, u.mobile2) as '연락처'
from user_table as u
inner join buy_table as b
on u.userID = b.userID"""

cur.execute(sql_01)
rows = cur.fetchall() # 모든 데이터를 가져옴
df_01 = pd.DataFrame(rows, columns=['userName', 'prodName', 'addr', '연락처'])
df_01

,userName,prodName,addr,연락처
0,강호동,운동화,경북,01122222222
1,강호동,노트북,경북,01122222222
2,김용만,모니터,서울,01044444444
3,박수홍,모니터,서울,01000000000
4,강호동,청바지,경북,01122222222
5,박수홍,메모리,서울,01000000000
6,김제동,책,경남,
7,이휘재,책,경기,01188888888
8,이휘재,청바지,경기,01188888888
9,박수홍,운동화,서울,01000000000


userID가 KYM인 사람이 구매한 물건과 회원 정보 출력

In [11]:
sql_02 = """select userID, userName, birthYear, addr, concat(u.mobile1, u.mobile2)
from user_table as u
where userID = 'KYM'"""

cur.execute(sql_02)
rows = cur.fetchall() # 모든 데이터를 가져옴
df_02 = pd.DataFrame(rows, columns=['userID','userName', 'birthYear', 'addr', 'concat(u.mobile1, u.mobile2)'])
df_02

,userID,userName,birthYear,addr,"concat(u.mobile1, u.mobile2)"
0,KYM,김용만,1972,서울,01044444444


전체 회원이 구매한 목록을 회원 아이디 순으로 정렬

In [12]:
sql_03 = """select b.userID, u.userName, b.prodName, u.addr, concat(u.mobile1, u.mobile2) as '연락'
from buy_table as b join user_table as u
on b.userID = u.userID
order by b.userID"""

cur.execute(sql_03)
rows = cur.fetchall() # 모든 데이터를 가져옴
df_03 = pd.DataFrame(rows, columns=['userID','userName', 'prodName', 'addr', '연락'])
df_03

,userID,userName,prodName,addr,연락
0,KHD,강호동,운동화,경북,01122222222
1,KHD,강호동,노트북,경북,01122222222
2,KHD,강호동,청바지,경북,01122222222
3,KJD,김제동,책,경남,
4,KYM,김용만,모니터,서울,01044444444
5,LHJ,이휘재,책,경기,01188888888
6,LHJ,이휘재,청바지,경기,01188888888
7,LHJ,이휘재,책,경기,01188888888
8,PSH,박수홍,모니터,서울,01000000000
9,PSH,박수홍,메모리,서울,01000000000


쇼핑몰에서 한 번이라도 구매한 기록이 있는 회원 정보를 회원 아이디 순으로 출력
(distinct 사용)

In [14]:
sql_04 = """select distinct b.userID, u.userName, u.addr
from buy_table as b join user_table as u
on b.userID = u.userID"""

cur.execute(sql_04)
rows = cur.fetchall() # 모든 데이터를 가져옴
df_04 = pd.DataFrame(rows, columns=['userID','userName', 'addr'])
df_04

,userID,userName,addr
0,KHD,강호동,경북
1,KYM,김용만,서울
2,PSH,박수홍,서울
3,KJD,김제동,경남
4,LHJ,이휘재,경기


쇼핑몰 회원 중에서 주소가 경북과 경남인 회원 정보를 회원 아이디 순으로 출력

In [15]:
sql_05 = """select userID, userName, addr, concat(mobile1, mobile2) as '연락'
from user_table
where addr = '경북' or addr = '경남'
order by userID"""

cur.execute(sql_05)
rows = cur.fetchall() # 모든 데이터를 가져옴
df_05 = pd.DataFrame(rows, columns=['userID','userName', 'addr', '연락'])
df_05

,userID,userName,addr,연락
0,KHD,강호동,경북,01122222222
1,KJD,김제동,경남,
2,LKK,이경규,경남,01899999999
